In [1]:
import geojson

import gaia

# Initialize Girder connection
apikey = 'bimEnIEt2RU8U1GqJKkqeJ5G5eH36kTSHGLjaUeA'
datastore = gaia.connect(apikey=apikey)


In [5]:
# Create gaia dataset for girder-hosted file
path = 'Public/DEM_bare_earth.tif'  # (~4GB)

# API option 1: create internal url
datastore_url = datastore.lookup_url('Public/DEM_bare_earth.tif')
#datastore_url = datastore.lookup_url('Public/TC_NG_SFBay_US_Geo.tif')
print('datastore url:', datastore_url)
dataset = gaia.create(datastore_url)

# API option 2: pass in tuple as the data source
# data_source = (datastore, 'Public/DEM_bare_earth.tif')
# dataset = gaia.create(data_source)

meta = dataset.get_metadata()
print()
import json
print(json.dumps(meta, sort_keys=True, indent=2))
print('Input dataset width: {}, height: {}'.format(meta['width'], meta['height']))
#gaia.show(dataset, representation='outline')
#gaia.show(dataset)

datastore url: girder://item/5bb7d7533b4e9a0007762163

{
  "affine": [
    322422.99999999907,
    0.5,
    0.0,
    4322541.000000004,
    0.0,
    -0.5
  ],
  "bandInfo": [
    {
      "colorInterpretation": "Gray",
      "dataType": "Float32",
      "noData": -3.4028234663852886e+38,
      "offset": 0.0,
      "scale": 1.0,
      "units": ""
    }
  ],
  "bands": 1,
  "bounds": {
    "coordinates": [
      [
        [
          -107.04646274467903,
          38.854612900051926
        ],
        [
          -107.04646274467903,
          39.03667206832321
        ],
        [
          -106.88737630031515,
          39.03667206832321
        ],
        [
          -106.88737630031515,
          38.854612900051926
        ],
        [
          -107.04646274467903,
          38.854612900051926
        ]
      ]
    ],
    "type": "Polygon"
  },
  "crs": "+proj=utm +zone=13 +ellps=GRS80 +units=m +no_defs ",
  "driver": "GeoTIFF",
  "height": 39816,
  "nativeBounds": {
    "bottom": 43

In [3]:
# Generate small crop geometry
bounds = meta.get('bounds',{}).get('coordinates')[0]
assert bounds, 'Dataset bounds missing'
# print()
# print(bounds)

# Compute center coordinates
x = (bounds[0][0] + bounds[2][0]) / 2.0
y = (bounds[0][1] + bounds[2][1]) / 2.0

# Use smll percentage of height & width
dx = 0.005 * (bounds[2][0] - bounds[0][0])
dy = 0.005 * (bounds[2][1] - bounds[0][1])
rect = [
    [x,y], [x, y-dy], [x-dx, y-dy], [x-dx, y], [x,y]
]
# print()
# print(rect)

# Must pass rectangle in as a LIST, in order to get geom formatted the way resgeodata uses
crop_geom = geojson.Polygon([rect])
print()
print(crop_geom)



{"coordinates": [[[-106.9669195224971, 38.94564248418757], [-106.9669195224971, 38.944732188346215], [-106.96771495471891, 38.944732188346215], [-106.96771495471891, 38.94564248418757], [-106.9669195224971, 38.94564248418757]]], "type": "Polygon"}


In [4]:
# Crop the dataset
from gaia import preprocess
cropped_dataset = preprocess.crop2(dataset, crop_geom, name='crop100m.tif')

Deleting existing (item crop100m.tif)
 1 Checking job 5bbcd5203b4e9a013ce517cf status... 2
 2 Checking job 5bbcd5203b4e9a013ce517cf status... 2
 3 Checking job 5bbcd5203b4e9a013ce517cf status... 2
 4 Checking job 5bbcd5203b4e9a013ce517cf status... 2
 5 Checking job 5bbcd5203b4e9a013ce517cf status... 2
 6 Checking job 5bbcd5203b4e9a013ce517cf status... 2
 7 Checking job 5bbcd5203b4e9a013ce517cf status... 2
 8 Checking job 5bbcd5203b4e9a013ce517cf status... 2
 9 Checking job 5bbcd5203b4e9a013ce517cf status... 2
10 Checking job 5bbcd5203b4e9a013ce517cf status... 2
11 Checking job 5bbcd5203b4e9a013ce517cf status... 2
12 Checking job 5bbcd5203b4e9a013ce517cf status... 2
13 Checking job 5bbcd5203b4e9a013ce517cf status... 3


In [5]:
print()
cropped_meta = cropped_dataset.get_metadata()
print('Cropped dataset width {}, height {}'.format(
    cropped_meta['width'], cropped_meta['height']))
#print(cropped_meta)


Cropped dataset width 143, height 207


In [6]:
# Todo Update gaia.show() for girder-resident datasets
scene = gaia.show(cropped_dataset)
scene

Using tile_url: http://localhost:8989/5bbbb6413b4e9a013ce517b6/{z}/{y}/{x}
tile_url http://localhost:8989/5bbbb6413b4e9a013ce517b6/{z}/{y}/{x}


Exception: Unrecognized layer type "tile"